# Learning with TensorFlow

In the rest of this chapter, we will cover the concepts that you need to learn basic machine learning models with TensorFlow. We will start by introducing the concept of toy datasets, and will explain how to create meaningful toy datasets using common Python libraries. Next, we will discuss new TensorFlow ideas such as placeholders, feed dictionaries, name scopes, optimizers, and gradients. The next section will show you how to use these concepts to train simple regression and classification models.

# Creating Toy Datasets

In this section, we will discuss how to create simple but meaningful synthetic datasets, or toy datasets, that we will use to train simple supervised classification and regression models.

# An (extremely) brief introduction to NumPy

We will make heavy use of NumPy in order to define useful toy datasets. NumPy is a Python package that allows for manipulation of tensors (called ndarrays in NumPy). Example 3-1 shows some basics.

Example 3-1. Some examples of basic NumPy usage

In [ ]:
import numpy as np
np.zeros((2,2))

Output:<br>`
array([[ 0., 0.],
       [ 0., 0.]])
        `

In [ ]:
np.eye(3)

Output:<br>`
array([[ 1., 0., 0.],
       [ 0., 1., 0.],
       [ 0., 0., 1.]])
        `

You may notice that NumPy ndarray manipulation looks remarkably similar to TensorFlow tensor manipulation. This similarity was purposefully designed by TensorFlow’s architects. Many key TensorFlow utility functions have similar arguments and forms to analogous functions in NumPy. For this purpose, we will not attempt to introduce NumPy in great depth, and will trust readers to use experimentation to work out NumPy usage. There are numerous online resources that provide tutorial introductions to NumPy.

# Why are toy datasets important?

In machine learning, it is often critical to learn to properly use toy datasets. Learning is challenging, and one of the most common mistakes beginners make is trying to learn nontrivial models on complex data too soon. These attempts often end in abject failure, and the would-be machine learner walks away dejected and convinced machine learning isn’t for them.

The real culprit here of course isn’t the student, but rather the fact that real-world datasets have many idiosyncrasies. Seasoned data scientists have learned that real-world datasets often require many clean-up and preprocessing transformations before becoming amenable to learning. Deep learning exacerbates this problem, since most deep learning models are notoriously sensitive to infelicities in data. Issues like a wide range of regression labels, or underlying strong noise patterns can throw off gradient-descent–based methods, even when other machine learning algorithms (such as random forests) would have no issues.

Luckily, it’s almost always possible to deal with these issues, but doing so can require considerable sophistication on the part of the data scientist. These sensitivity issues are perhaps the biggest roadblock to the commoditization of machine learning as a technology. We will go into depth on data clean-up strategies, but for the time being, we recommend a much simpler alternative: use toy datasets!

Toy datasets are critical for understanding learning algorithms. Given very simple synthetic datasets, it is trivial to gauge whether the algorithm has learned the correct rule. On more complex datasets, this judgment can be highly challenging. Consequently, for the remainder of this chapter, we will only use toy datasets as we cover the fundamentals of gradient-descent–based learning with TensorFlow. We will dive deep into case studies with real-world data in the following chapters.

# Adding noise with Gaussians

Earlier, we discussed discrete probability distributions as a tool for turning discrete choices into continuous values. We also alluded to the idea of a continuous probability distribution but didn’t dive into it.

Continuous probability distributions (more accurately known as probability density functions) are a useful mathematical tool for modeling random events that may have a range of outcomes. For our purposes, it is enough to think of probability density functions as a useful tool for modeling some measurement error in gathering data. The Gaussian distribution is widely used for noise modeling.

As Figure 3-5 shows, note that Gaussians can have different means μ and standard deviations σ . The mean of a Gaussian is the average value it takes, while the standard deviation is a measure of the spread around this average value. In general, adding a Gaussian random variable onto some quantity provides a structured way to fuzz the quantity by making it vary slighty. This is a very useful trick for coming up with nontrivial synthetic datasets.

> **Figure 3-5: _Illustrations of various Gaussian probability distributions with different means and standard deviations._**<br><img width="300" src="data:image/png;base64, ..." />

We quickly note that the Gaussian distribution is also called the Normal distribution. A Gaussian with mean μ and standard deviation σ is written N μ, σ . This shorthand notation is convenient, and we will use it many times in the coming chapters.

# Toy regression datasets

The simplest form of linear regression is learning the parameters for a one-dimensional line. Suppose that our datapoints x are one-dimensional. Then suppose that real-valued labels y are generated by a linear rule

$$ y = w x + b $$

Here, w, b are the learnable parameters that must be estimated from data by gradient descent. In order to test that we can learn these parameters with TensorFlow, we will generate an artificial dataset consisting of points upon a straight line. To make the learning challenge a little more difficult, we will add a small amount of Gaussian noise to the dataset.

Let’s write down the equation for our line perturbed by a small amount of Gaussian noise:

$$ y = w x + b + N(0, \epsilon) $$

Here � is the standard deviation of the noise term. We can then use NumPy to generate an artificial dataset drawn from this distribution, as shown in Example 3-2.

Example 3-2. Using NumPy to sample an artificial dataset

In [ ]:
# Generate synthetic data
N = 100
w_true = 5
b_true = 2
noise_scale = .1
x_np = np.random.rand(N, 1)
noise = np.random.normal(scale=noise_scale, size=(N, 1))
# Convert shape of y_np to (N,)
y_np = np.reshape(w_true * x_np + b_true + noise, (-1))

We plot this dataset using Matplotlib in Figure 3-6. (you can find the code in the GitHub repo associated with this book to see the exact plotting code) to verify that synthetic data looks reasonable. As expected, the data distribution is a straight line, with a small amount of measurement error.

> **Figure 3-6: _Plot of the toy regression data distribution._**<br><img width="300" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAoAAAAHgCAIAAAC6s0uzAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3df3Ab533n8QfUD9JmRUCQxiHtyrGWVjphVVcGaTdqOmlPAuY09jnutYDkuO7M9XICosQ3qdM5ouzUY3PcGQZMe0nv2rqA0klnqvqHgGtO1dijCdbpXMY+pRIBK6nKzCQHULFOFtsTQZCOZNGKiPvjsdZrYHexAEES2H2//qKIX8IuuB88v76Pp1KpCAAAsLa6OAQAABDAAAAQwAAAgAAGAIAABgAABDAAAAQwAAAggAEAIIABAAABDAAAAQwAAAEMAAAIYAAACGAAAEAAAwBAAAMAAAIYAAACGAAAEMAAABDAAAAQwAAAgAAGAIAABgAABDAAAAQwAAAggAEAIIABAAABDAAAAQwAAAEMAAAIYAAACGAAAEAAAwBAAAMAAAIYANqUqqrFYrGFT5hKpTiqBDAANKxcLrvnzRaLxWKxqChKC58zGAySwQQw1l8qlfKYi8fjK3nyeDy+detWw5uGh4cjkciavc3JyUn9+xocHFzhW2uV1h6Htn2brWoIyve1devWwcHB1jYKzY7n5ORkJBJZx8OYSCSi0aj+D1aanJy0f9yqPmOKopTLZVd9jyGA0Y6i0ej8LblcTgiRTqe13yQSiVV63VgsdujQobV8pz6fr3JLNpstFovDw8Prfvxbfhz0bzOdTququjZZtTYN31wul8vlCoVCoVBobaPQ8Ovj6Ojo6OhoOp0uFost/J6Uz+djsZjH44nFYvp3F4lEtm7dGovFtGiUp0+fvuVyORqNRqNRRVHqfi1QVTUej8v/f9VN4XDYSV/OXKECRysUCkKIbDbbqiccHR3V58E6SiQStf8Tn8+XTqeddAYN32YwGAwGg3Ufm06nFUVp53eXTqcLhUKrni0ajVrcOj8/HwgE5ufn5T/ld9O6r279nHrZbHZ0dLTqojo/P59MJqueUPs/VCoVRVH0/webf1zpdDoQCNT+3s6nAu2DFjCcxhlNw7p9mKqqqqrK6W70s6F9PGRru4WflmKxKL8t6cdiVVU9ePBgVRPW5/NpTeSqweByuZzP55v+PyiKspKHgy5orLp4PD44OCh7xrRe01AopO9P83g8jT5tKBTSesBCodDk5GQsFpNje/qo0H6pXacymczw8LAc4MxkMvonnJycTKVSVc9g1pkZiUTK5XI4HDZ7oXK5HAqFPB7P8PCwPAiyy9rwhWofLh8i/5/ybrW/qToOhkfb+vjYEQgEFEXJZrMWBzAUCkUikWKxKAdZZdKYHep1pKpqJpNJpVKr3X3q8/lkI1j/Ra3lnd7RaFQ/ylMqlbS4lS+qf8Xa+Pf5fCv5TjA8PMzXsg6ykUPgNvKiLK/d8Xg8FApls1kZEtrVIZlM6ieJNB3z6XQ6Ho/H4/FYLCY7w2VG5nI5OQI3MjISCARKpdLRo0cVRZmamgqFQvqxwGQyKRt8wWDQMHTljDBtgE12upq9UCQSkT1+mUxmYmJiZmZGuzhWvVDtw0ulUiqVyuVyfr9/ampKJkfVb2webYvj01BbR7tSGx7AbDabyWTi8bj+mS0OtZlMJlMqlcrlsuxfLZfL+/fvl/23K6coiqIoMhRTqVQsFpMnQr6uECKbzcZisampqVwuF4lEDD8GzUkmk8FgsIUB7Pf75fe2ycnJfD6vJX3VJ1b/iqVSqfZJan/Z0P+hVacGa4FeeFeNAct/6oegfD6fvDUYDI6OjspfCiFyuVyjY8D6ZwgGg9oYlYyc2leXMV/1JIFAQBszCwaDstViMTgqZ5al02n9M5u9kP59Cd34X9ULGT68djzVbIRVOw7WR7v2+NgfA65UKuFw2HDAT38ArceA9fc0k7tFe55kMhkOhw3/PxbqvlDV4dIG8qPRqHy50dFRs89kE+O18h2ZfbSaeM5sNqs9WzAYlP/n+fn5qukX6XRa/4S1p15RFDvHymwMOJvNGp4dtCdawO6iqqqiKPo+sZGRkWw2GwwGY7HY4cOHE4lEJpPRGiUrMTIyom8ZyJ5tIYR+FZPWoJmcnHz55ZflkJi+xSaj0bpfUQgRDocDgcDx48dlw93shcLh8Msvv6woihyH07dF9C9k+HD5DB6PRx4rmX9Vv7F/tA2PT0Py+bz+Fc0OYC3795SHV07N1V4rm83qRys08jvQClvDslc2EAhoH7+pqSnZo2s4e79qGrOqqvrmYygUMuvIicfjuVzO8KPV3HMWi0Xtwyy7OsrlsqqqtZ+K2kazXqlUau7zALqg0cHC4fDhw4dVVZWdfit/QsOrWyAQqO0iGx4eVhTl6NGjgUCgah2R/R7CRCIRiUQOHjwoX9fwhWRu7dy5U1GU1157zeKFDB8u+65ffvnlSCSSSCRGR0drf7PC42OTnEykBaHFAbR/qC1CUfa0a4HUqpVs5XJ5586dsjEqPlyLQzsd+Xzeotu5KvX1PdgW5N3Mjn9zz6kne7ZTqVTtS/h8Pn2ca29cPy1rJb3iK3w4CGCsopGREX0VnnK5PDU1pcVtNBrNZrPHjx+fmZlZjVcPBAL5fF5/uZFZks/nVz5wFQwGR0ZG4vF4Mpk0fCF5Nbcz1Gr2cPk1JRwOp1KpZDIp47b2NzaP9krE4/FAICCTyf4BbO5Q6ycryb4Bw0u89Zpas4bjyMiI9mzyhfRdL6qq6udMtSRa5Gwv/dtZeWdPuVyuarbGYrGJiYnabyqyXEZVB0PVRK2V/H9KpdK2bdu40HUKZkG7SyAQCIfDkUgkn8/L6UWKomi9ZGNjY6lUamRkpG7jrPxhNl9dUZRoNLp//34Zb/pL4eTkpPzNShZRJBKJVColr9S1L6Rdf1VVlb9v6P8pJzTJQM1ms4qi1P6moaPdnHw+HwqF8vm81lCzOIByolY+n89kMlqAGd6zWCyaFTLUN6pefvnlqkU1+oajBcP09fl8+t5sLbHy+bw8X8lkUuuob8nqmkwmI/ve5dTrZDLZklA/fvx4VTM9Go0aNkZl3FZ9l9Ja2KlUSstsizMijGZvSblcbuXfJ7B2GAZ3YSEOOZHK5/MZzoGyLmRh2Msqp59UTcLSfpZNrtpXD4fDchqUvOjI2AsEAoZPYnN2kn5qUu0LVV0lA4GA9vvaF6p6eC6XC4fD8pIaDAZllbGq3xj+tw2PtsXxqX2btV8OqmYPmR1AeUJ9Pl8wGLQ+1LJL1mIiUjqdlnHV8joniURClmbUJh9ls1nZGVMoFLSXbkkhjiYugHWfMxqNyg9J1Ukxe2DtPROJRDKZlEMY2i/lGamdJpbL5UZHR+WnTn/QKMTRiQhgfOhq0iZVrlaD7JrWT3UOBAL25806/tQbzqrtrI/HapzNlj+n/e8TsjxnQ9+2+Tx3Frqg8UGXcjweX/ny37bl9/uLxeLU1JTsfJZvWV+V181UVTUcnJYrreURO3z48NGjR9v5XaxGKY+WP2c4HLY5DC9n0dt/5mQySS3ozuKRCyLhcplMJhKJBAKB1157bSWzc9v/bU5MTGhTb2KxmIO/cLRKKpWS3120aV9YITkIbf3Zk1P27B9wO88JAhgA6HJQZRWwFn5PIn0JYAAAUB9jwAAAEMAAABDAAABglbirFOXy8vLbb7+9ZcuWJja7BQC0j0ql8s4779x5551dXZ3aknRXAL/99ts7duzggwsAznDx4sWf/dmfJYA7wJYtW+QJ6+vr44MLAJ1rcXFxx44d8qpOAHcA2fPc19dHAAOAY67qHYpJWAAAEMAAABDAAACAAAYAgAAGAAAEMAAABDAAACCAAQAggAEAgBMCeHBw0OPxbL1lcnKSkwoAne7mcuV0Ye7EuUunC3M3lyuOfI8dX4oyFouFw2G/3y+EmJiYGB0d5YMLAB3t1PnL4yenLy9cl/8c8PY888jQgd0DDnubnkqlg79ZlMtlIYTP5xNCqKqqKIqiKBb3X1xc9Hq9CwsL1IIGgLZN3yPH8vpkkuWen38ioM9gB1zPO7sL2ufzyfQVQuTzecP0XVpaWtThww0AbevmcmX85HRVu1D+c/zktMP6oh0yCWtycjIajRreNDEx4b2FzYABoJ2dmSlpPc9VGXx54fqZmZK4NTz8yvff7vQ329ld0Jrh4eFcLmd409LS0tLSkvxZ7h9JFzQAtKcT5y598aVzZrf+6WN7ujd2yeHh5aVrF792sKOv507YD1hVVYtbu7u7u7u7+VgDQPvb/jNWl+sLV659Tf2hY7qhnRDA+XxezoIGAHSuU+cvP/v3/2x4k0eIj/R1v3jmLScNAjthDNjn81lPfgYAtH/6HjmWn11cMkxfIcRnHrx7dvG6k96yE1rAZtOvAAAdwXDys6bf2/PMI0NLP1122LumFCUAYJ3T96/fmDGc/Cz9cfgXD+weuGNLj8Pe+EbOPQBgvVQVvTJ05eqSEOLBnf4Bb8/swnXHDAPTAgYArFv6HjmWt05fIYRs+27o8jzzyJC4NSRMAAMA0AzrcV+N7/ZND+58f53Lgd0Dzz8R6Pc6pC+aLmgAwDowK3pVpXztRnZ6VqsCfWD3QGio/8xM6cLl//f412gBAwDQoH99x9aaIk9NFegNXZ69g9sevu/OTj8CBDAAYB3YnNWsrwLtMAQwAGAdyFnNNmdU2WwuE8AAANTR0Kxm5y0CJoABAOvGzqxmjxAD3h5tIrSTMAsaALCeGSxnNf/rO9flZkdCiIoufYUQzzwytKHL47z3TgADANaTnNUsf/65/p/RF8aSVaC1NUgEMAAAq94gvmNLz4M7/Y5s+xLAAIDVdXO50mia6hvEzkYAAwBWRdVGCwOO7k9uArOgAQCrkr5VGy3MLlw/cix/6vxlDg4BDABYFYYbLch/VtWVJIABAGgZs40WHFxXkgAGAKw/68qRjqwrSQADANafdeVIR9aVJIABAOvPbKMFB9eVJIABAOvPcKMFZ9eVJIABAG2hdqOFfm/P808EWAesoRAHAGC1Mtg9dSUJYABAk5ooG1mXe+pKEsAAgGZQNnLtMQYMAKQvZSMJYADA2qJs5HqhCxoAXM26bORfvzGzfUs3U6gIYABAC5q8+slW1oUhn3vlB/IHRoUJYABA82onWz32wN12HihHhVnI20KeSsVF/fuLi4ter3dhYaGvr49zD8BtTd4LV659Tf2h/qIv+5S9t28qX7tRPzCE6Pf2vB7f1w590Q64ntMCBgAXNXmrVGQG22uJaZsJsrSXAAYA1EnfI8fylXqxWn73p/afk80EW4VlSADgTIbri1aOzQRpAQMArJitL2qaHANmM0FawAAAK63tK2YzQVrAAABbWttX3M86YAIYAGDHgzv9A96e2YXrFsPA/t5NpaumC5B+d/+uX1K2sZkgAQwAaMCGLs+nf3Eg+Z2Z2pueCu66Z3vvHVt6hj+69Ve/8g+GId3f1/2f9+8idAlgAEBjTp2/nDJK39indn4x+DHtn888MnTkWN5zawMGcWu499lP/zzpu6qYhAUADmSxBunvv3dZv8fRgd0Dzz8R6Pd+MGDc7+2h5CQtYABAMyzWINVWszqweyA01K/foYG2LwEMAGiG9Rqk2ls3dHkoMLnG6IIGAAeyXoNENSsCGACwKuQapNp+ZI8QA1SzIoABAKtkQ5fnmUeGxK0pzVr6CqpZEcAAgFXF9OY2xyQsAHByBhtOb765XGHOMwEMAHjfauRi7fTmU+cvj5+c1hYpDVDkmQAGADdbm1w8df7ykWN5fYGO2YXrR47l6Zpee4wBA0BbpO+RY3l96QyZi6fOX9Yax6cLcyfOXTpdmNPXsbJzq/5uteWx5D/HT05bPBC0gAHAgcxy0SPE+Mnp0FB/dnrWonFsv+lsVh6rYlQeC7SAAcDhrHPxz779I4vGcd2ms16j5bFAAAOAk1kn3zfeuGDWafzeT5cb6lKmPBYBDACwm3zld2+YNY7/5vQF6y7lqt9THosABgB8KBf7+wwy2COE7/ZNFg/8celaQw1rymMRwACAD2SnZ6//9GZt+gohfueXd1o88KP+2xttWFMeq304YRZ0JpMplUp+v18IEQ6HOakAOkjtwlzJd/umid/4hdBQ/0tn35pduF6pied+b89v773n66/PmN1q1qXM7r+0gFsjlUoVi8VoNBoIBOLxOGcUQAcxXIAkdW/sCg31W3cab97Y1VyXsiyP9eieu/YObiN914unUunslddbt26dn5+XP5fLZZ/PZ3HnxcVFr9e7sLDQ19fHuQew7k4X5j5z9Ltmtz798Me3b+m+Y0vP/NX3nnulBeuAncQB1/PO7oLO5/NCCFVVhRDZbDYWi9UG8NLS0tLSknbC+IMH0D6sFyA998oPtEx9+uGPb+3tNuw0pkuZAF4HU1NT5XJZURRFUUZGRoaHhwuFQtV9JiYmxsfHOdMA2pDNpbezC9e/8MKbzz8ReHTPXYZ3qN1xAe2vs8eAFUXx+XyKogghfD5fsViUbWK9sbGxhVsuXrzIKQfQPswW5lahXDMB3I4BXC6Xre/T3d3dp8MpB9A+DOdYmWWwYW0NEMDrFsCBQKBYLAohisWi/CcnFUAHqV2Ya4FyzU7S8euA0+l0MpkcHBzM5XLZbJYzCqATM1ibRXXlnSVt7lUtyjU7SccvQ2oIy5AAtLmby5VfSXzbrLbG6/F9zHB2zPWcUpQAsA4pe7owd+LcpdOFuap5VZRrdo+NHAIAaDQ+V7Lotm7dDDkqrL9Pvztqa7gNXdAA0Mr4rPvwqsrPHiEqQjwV3HXP9l59oq8w5rmeE8CcMACOSt/a+BRC2NxNSI7vGu7gqxmgseua6zljwABgi+HGCQ2VyDgzU7JOXyHE7ML1I8fyp85f5oA7HgEMALaYxWdViQyLCVZ2VvFS9Mo9mIQFALZYx6e81XqE2OYqXi3RKe9MCxgAUCc+79jSI0eI9a3kqv5km5Wf7TeXQQADgPOZxadHiAFvz/BHt9YdIbZf+VlQ9IoABgBI1iUycj+etzNCbKfys0z0B3f6OebOxhgwANhlUSLjxLlLFg/U9yfrKz9fuHLta+oPtbayoOgVAQwAMMtgLT71JTLqjhBXNaa1CVY/1/8zFL0igAEA9enjUyNHiM02UbDoTzZLdBDAAOAcq1ffUY4QHzmW9zTen2yY6CCAAcAhVljGuS42UUBDqAUNwC3pu5Iyzu3QyIbDrue0gAE4UFUKCiEMF+l6hBg/OR0a6tfH5AoTlP5kEMAA3NvYrepqfuyBu60X6WqRudrd1ICGQhwAnJa+tfUgv6r+0OIh2iLdurUkAVrAAGDAYsdAC3KRrtljZTf1lu5NV64uMawLAhgADNjZcFdPv0j3u8U5i27q3/qrf5T/pFMarUIXNADnqLuDkGEZ5w1dnlPnL3/hb/N2XoJOaRDAAFDNuh7kU8Fd+l0Q+r09cg2SHPotv3vDzktUbXBk6OZy5XRh7sS5S6cLcxZ3g8vRBQ3AOazrQT65b9eT+3ZVLTEyHPqtm8FVc6f1mEcNWsAAXMd6x8ANXR65SPfRPXftHdwm51I1OmysMezuZh41CGAALlW74a7W1Ww/R+2o7e62mINt3WUNd6ILGoADM9j+/kLWw8aGzDY4MmtMW3dZgwAGAOeoqgdpUV3SbNjYIn2FyQZH1o3pppvaIIABoCNZz4oy3EbQgsUGR9aN6Saa2nA2xoABODx9686Kqh02HvD2xD61c0D3G3/vps9+8p4XD3/i9fg+s+Fk2Zj2GDWaB4y6rOFybEcIwLFuLld+JfHt2nFZOYj7enyf9SZITWyLJPNe6BrTq7TpIdiOEADaV0Ozomq3EWxiY0HZmNb3ePezDhgEMAC3WZdZUQ3NwQYBDAAOtF6zoppoOsOFmIQFwLGYFQUCGADWx2MP7KitCy1MFvICa4kuaACdodE5yVXLfzXMigIBDAB2NbrFkFwOVLvI8qngx57cdy9tX7QDuqABdED6NrTFkNkOgx4hXjr7FscTBDAA1HdzufL7f/dPDW0xVHf5L0cVBDAAfBC0pwtzJ85dOl2Y08fqn337R+VrNxpKUzZFQEdgDBjA+jMb4r25XPnGGxcaTVM2RQAtYACwlb5mQ7xnZkrld280mqYs/wUBDAB1GE6Y0oZ4Zxetuot9t28yTFO5w6C4teRXS1/B8l8QwAAgWU+YKv1kyeKxv/PLO83StHaHwX5vD1sSoa0wBgxgPVlPifL3bh7w9swuXK8YNX+f3HevxWPZFAEEMACYsp4S1e+97ZlHho4cy3t0O+xKX/6NX6gbqGyKgHZGFzSA9VR3wlRtZ/KAt+cv6UwGLWAAWAk5YaqqjVs1YYrOZDiSp1KpuOfdLi4uer3ehYWFvr4+zj3QPhoq9dzorgzgek4LGACM2W/jNrorA0ALmG9MgKOsSzO0do8j+aqsL6IFTAsYgCusSzPUrGSHR4jxk9OhoX76otFZmAUNoJlmqP3NAc3StHbrBbP9GCT2OILD0AIGsNbNUMMGtBDCulXNHkegBQzAvVbeDDVsQH/uWP5z9VrV7HEEAhiAe62wGWqx9YIw+uX4yWmtL5o9jkAAt5dYLObxeDwez/DwcD6f54wCq2qFzVCzBrQwyWB9q5o9juAwHT8GPDg4OD8/L4Tw+XycTmC1yWZo7e4IHiH6bTRDmxip1T9ElqXUDxX3sw4YBPA6InqBNWOncuRKmsi1tvd26/9JWUo4Rsd3Qc/NzWUyGVVV4/F4sVjkjAKrremtdm8uV5aXK77bNjX2eh6DLwF7B7c9uueuvYPbSF/QAl43sVhMURQhhN/vj0QiuVyu6g5LS0tLS+/v6b24uMgpB1qSwY02Q6uWHtl35SdLHHAQwO1Ipq/8IZ/Pl8vlqh7piYmJ8fFxzjTQWg1ttVtbQlLP37updPWG2WNZXwSn6uwu6Hw+v3XrVvmz2Ujw2NjYwi0XL17klANrRla2+uabl/7gm/9kmL6+2zb97X/6pe+OBVlfBFrAndf8HRsbkz9nMplgMFgbw93d3d3d3ZxpYPVS1rAv2k6fc/ndG10ez+aNXSuZ2AUQwOvA5/MFAoHJyUmfz1coFNLpNGcUWEtmuzJY9znryVVGrC+CC7EdIYDm09dwc8A/f/z+5175gc35Vi8e/oQ2lrwuWxyC6zktYACdxGJXhj88cd5iUpVef1+3foi3oYldQKejFjSAZljsymAzfYUQn3nwbpq5IIABoAEt2f7vnu29HEkQwADQAOvluf7ezZ4VPwlAAANANevNAf/o0d3CoIhk9d1Y4wsCGAAaY7054EP3VdeLFkZ3YwAYbsYyJADNr/8xWwdc9bQXrlx78cxbs4vGdwPceT0ngAG3sw7RVoU3a3zB9ZwAJoCBD6WvYTENO9sLAlzPV4IxYMC9zIppCCHGT07fXK5wiAACGEDrWRTTuLxw/cxMScvp04W5E+cunS7MkcpAq1CKEnAv62Ia8tZXv/+2vrQk86cAWsAAVsq6DsYdW3omXp3+/Atv6ktLXl64fuRY/tT5yxw9gAAG0CTrYhpXfrKU/M5M7aMqjBADBDCAlTAspiEj9uDIjmf+/p/NHqgfIQZAAANo2IHdxiWr/vS1H5WuvmfxwJZsxgAQwABcpGpW84HdA6/H9z0V3NXQk7CPArBCzIIG3MWw7lVoqP+lsxftP4m/dxP7KAAEMOD8BmurijjW1r2aXbh+5Fj+d4MfM1wQbOaPHt1NLUmAAAZc12BtbhmuWd0rjxDf+N8z9p8n9qmdD913J6cGWCHGgIG2Tt8jx/L6tunsCpbhWtS9Kl+7YecZtvVu/ovHA2MPDXFqAFrAgGNZNFjHT06HhvrtdALru69nF95t9P/gEeIjfd1/cnDPlZ8ssYsRQAADrlC3UPPewW11G9D67mt/7+ZG01cI8eynf/6T927ndAAEMOAWdgo1W6dv1Xwr63W9tfop+wwQwIALWS+03d7bfbowZzY12rD7uiFPP/zx//DJnXQ4Ay4KYL/ff+jQoUgksm/fPk4P3EwWap5duF6Vox4hvLdv+r3092YXTadGm3Vf27d9SzfpC6yqtpsFPTMzEwgEvvzlL+/atWtsbOzChQucJLiTYaFmz61Jy1r6CqOp0SuvE0mhK8B1Aez1eg8fPvytb31rampKUZRIJPLAAw98/etfX1xc5GzBbWoLNffdttFT0y6VTWT9DkUriU+5FRKFroDV1qZjwOfOnUsmk9lsNhAIxGKxcrkcDocPHTr02c9+lnMGt2VwaKhfLiW6cOXaV9UfGt6tamq0Wfe1WeJWdD8LIZ55ZIj+Z8B1LeCvfOUr9957byQSGRwczOVyx48f379//2/+5m9+61vfGh0d5YTBhTZ0efYObvt399350tm3rO+p9Tyb7TNY29L9i8fv17ew+709zz8RYOYz4MYWcKFQSKfT999/f+1NX/7ylzlhcC0786r0Pc+y+1q/DrgqfWVL98DugX+7e6BVtaYB2OepVCruebeLi4ter3dhYaGvr49zj85y4tylL750zuIOA96e1+P7atcj3eq+vvrimbdmF5e0O7PGF1zPaQEDqK/uvCrDgVvZfS1/fnLfLlq6AAEMoDHz5nWsujzizz5Tf+BWH8YACGDAXZrb3PfmcuW5V6bNbv1vh/Y8dB+dyQABDMDEq99/+w9PnC9dfX/vP/sDsdYzsLZRNAPoQOwHDKyRiVenP//Cm1r6CiEu297cd4UbMwAggAHXtn0vJ78zU/v7yocrWJmxnoFF2UiAAAZg4OZy5Q9PnDe7VVawsn4GWdmqdriYspEAAQzA1JmZkvVevHX7kM02ZhCUjQQIYABN56udPuTajRkoGwl0NGZBA6vOOl/9vZts9iHrN2agmAZAAAOoQ47gmq0j+qNHd9vPUYppAI5BFzSw6uQIrmHGxj6186H77uQQAQQwgFUhR3AHdCO423o3/8XjgbGHhjg4gDvRBQ2sXQYzgguAAAbWASO4ADR0QQMAQAADAOAOdEEDrdHcPoMACGAAzTt1/vL4yWltpa/9fQYBuBZd0EAL0vfIsby+zj8+AkUAABQ1SURBVMas7X0GARDAAJpxc7kyfnK6ajdB+U87+wwCIIABNOPMTMmwxmTF3j6DAAhgAM2w3umo7j5IAAhgAM2w3unIzj6DAAhgAA2TOx3VLjnyCDHg7bG5zyAAAriDxWKxcrnMGcUakzsdGd70zCNDrAYG4PAAVlX1+PHjpRITXrAODuweiH5qpz5quzwi+qmdrAMG4PAAlg1fv5++PrTezeXK6cLciXOXThfmzNYUnTp/OfWdGf2NlYpIfWeGdcAALDihEtbx48ej0SjnEi1np76V2TpgjxDjJ6dDQ/30QgNwZgtYVdWDBw9a3GFpaWlRh1MO++lrp74V64ABuDGAZeezz+ezuM/ExIT3lh07dnDKYYf9+lasAwbgxgBWVbVYLKZSqVQqVSwWM5lMPp+vus/Y2NjCLRcvXuSUww777VrWAQNoTmePAYfDYe3nWCwWDocVRam6T3d3d3d3N2caDbHfrpXrgGcXrlc1lz1C9LMOGIBTW8BSuVyenJwUQiQSiWKxyElFQwznOVu3XC9cuab9rK0D9nw4fQXrgAFY8lQqLtqtZXFx0ev1Liws9PX1ce4hzOc531yu/Eri27XtWi1fn38ioJ8OzX7AANdzApgARgPpe+RYvlLTcpXheur85c8dywuTAO739rwe36dv4N5crpyZKf3rO9fv2NLz4E4/bV+A67k1akHDperOcz6we+Cp4C7DxxouMdrQ5dk7uO3RPXftHdxG+gIggAFjduY537O91+IZWGIEgAAGGmZnnjNLjAAQwECL2QlXthoEQAADjam7iYKdcGWJEYDVs5FDAOexuSjosQd2fFX9kbAM1wO7B55/IqB/tn6WGAFoBZYhwYHpa7G4yDChNWZRzRIjgOs5LWDAip3NAWsTWnoq+LEn991LuAIggIGG1V1c9OBOf21Cy1byi2d+PHLP1is/Wapq5lLlCgABDNRRd3GRRULPLi791tf/sSpla5vLclfgqlKUANAoZkHDUeouLrJZPUOm7Kvfv2xzV2AAIIDhanUXF9msniGj9ekT523uCgwABDBcre7KXbOENkzZuavvWdyBUpQACGDgA3Llbr/3g5Zuv7dHG7I1TOjmUIoSwEqwDhjOZL1y12wdcC1/76b5qzeq/kgMtyMEwPW8IcyChjPJzQEtWsmhoX6Z0Nt7u38v/b1/WbxumLJPP/zxL7zwpufWqLCgFCWAFqELGq5O6Ef33PXJXduf/bTpsPFD991p0aENAE2jCxoQol61DUpRAlzPCWACGKuFlAW4nq8lxoCB91kPGwNAazEGDAAAAQwAAAEMAABWCWPAaEfMhwJAAANrjf13AbgBXdBou/Q9ciyvLxIpdwY8df4yBwcAAQy0xs3lyunC3Ilzl04X5m4uV24uV9h/F4BL0AWN9WzsVnU1P/bA3db777JOFwABDKw0fY8cy+ubtLML17+q/tDiIey/C8BJ6ILGOrDoarbA/rsAaAEDK3JmpmRnL16N3BnwwZ1+Dh0AWsBA8+p2JhvuDMhqYAAEMLAi1p3JTwV3sf8uAMejCxrr4MGd/gFvz+zC9UpNw7ff2/Pkvl1P7ttFJSwABDDQYhu6PM88MnTkWN6jm3tV1dXMiiMAzkYXNNbHgd0Dzz8RoKsZAC1gYB0yODTUT1czAAIYWGsbujx0NQMggAFT7A8IAAQw1hr7AwJAyzEJC/XTt7n9Aat2OuJIAgAtYNhlVrTZI8T4yenQUL9ZXzSNZgCgBYzmmRVt1vYHbG2jGQAIYECIekWbDW+12Olo/OQ0fdEAQADj/by0GKm1LtpseGtzjWYAcBvGgF2t7kitddFmw/0Bm2g0AwAtYLgrfeuO1MqizaKR/QGbaDQDAAEMt7A/Utto0WbZaK5NZo8QAyaNZgBwIbqgXaruSK2+QmRDRZvt7HQEACCAXarRkVr7RZtvLle8t23+j5+855vnLpWu3tAazawDBgACGKs1Uls1q8vfu/nX99wZGuqnfDQAVGEM2KVWY6S2dlbX/NX3vvHGhYV33yN9AYAAhhBNTW+2Rv0NACCAYYvh9OY/f/x+722bm9hBgfobANAQxoDdnsH66c3zV9977pUmd1Cg/gYA0AJGA+T05kf33LXw7ntfeKH5HRSovwEABDAatvIRXOpvAAABjIY1NIJruH9Dy2d1AYCzdfwYsKqq5XK5VCpls9mxsbFAIMBJbYL9EVyL/RvkrC79rdTfAADHBnAoFMrlcuFwWAgRiUQKhQIntQk2R3DlSl99f7QcJ9ZKQzdUtBIACOAOVigUFEWRP/t8Ps5oc+QIrmEvtBBi/uqSMB8n9ggxfnI6NNQvs9Z+0UoAcLOOHwPW0rdQKKTT6do7LC0tLepwyg1t6PI8/fCQ2a3PvfKDm8sVVvoCAAH8IcViMR6P5/P5crlce+vExIT3lh07dnDKDadQCSG29m42e4jMV1b6AkALOaEQh6IoiURicnJy//79MzMzVR3RY2NjX/rSl+TPi4uLLs9giylUdfOVlb4AQAv4Q21f+XM4HC6Xy1NTU1X36e7u7tNxZ+tWS9+qzRL0pTbq5isrfQGAFvAHAayqqvazz+cbGRlxcLhazy62aN0KG1OoZL7OLlyv1ORrv/f9V3zmkaEjx/KeWzU6BCt9AaBZnkqls7epSaVS8gc764AXFxe9Xu/CwkLHNYWtw1UI8er3L3/+hXxVcAohtAVCpwtznzn6XbPnf/HwJ/YObpNNZFGTr9qT2PmfAMAa6NzruXMC2A0nrHb1bVUuvvr9t5988c3aepGy8fp6fN+GLs+Jc5e++NI5s5f408f2PLrnLpv5WrctDgBcz+tiN6R2V7frODs9+/kX3jR8rLZAaO/gNptTqOxU0mClLwAQwM5nvfr2u8W58ZPT1s8gpzebDfEKIbo875faIF8BYM2wGUO7s14ddLowZ1a+qqp1q22WUGu5Ir7wwps2tx0EABDArlBvfW2dIfxtvZtnF96Vq5IO7B7488cDZiO2NrcdBAC0BF3Q7c6i63hb72aPp84EqLmr7z11/Hvi1nSqrb2bDUNWP1rMMQcAWsAw3mdXC9f//u3/Y/N5ZM0NdXrW4j6UkwQAAhgfkPvs9ntXVOtRtnu/ee6SxX0oJwkABDCqM/h//Zd/4+/dtMIMLl294e/dTDlJACCAYVfux/OlqzdW/jy/vudO8eEObcpJAgABjA9U7azQqgHa0FD/808EPtLXrf3mI33d+mKTAIA1wCzoNlVbEvKxB+xupDjg7alUKv+yuGS2rUJ2etaoDQwAWDvUgm7T9K0t/lwRwnf7poVrN+qesL98IiCEMNtWQd5kUVkaALierwG6oNuORfFn+2onTvd7e55/IhAa6jd8ckEhDgBYW3RBtx2L4s/lazeeCn7shX+88C/vvGfxDHKTBsNtFcxKV1KIAwBoAbuopaufY6X93nqy1T3bb/+vh+63fmYZpeLWtgqP7rlr7+A2OcPZ+skpxAEAtIAdzmLb3br7BtqJSbP72NyUEABAC9iZ6XvkWF7fFSzrRMr9iOavmnYvy1oZdmLS7D6ysjSFOACAAHYdszlWQojxk9Pv/XT5uVdM9/d9+uGPb+jymIVoVU4b3mRYWZpCHABAADufxRyrywvX/+b0BYv9fbf2dgvLnX1lmlpHqdkEadYgAcBaYgx4rVmP4P64dM3OY2WI6keRtbavNpZswXCCNKcGAAhgJ7Mewf2o/3abj9VCdHbxeuknS/7ezf3e2+xHqZwgzekAAALYLeQI7uzCdcM6kb+9956vvz5jdmvVyC4hCgCdizHgtWY9DWrzxi4mSQEAAYxVYT0NiklSAOAGbMawbm4uVyymQVnfCgAu54DNGBgDXjfWI7iM7wKAs9EFDQAAAQwAgDvQBb3WGNwFABDAa81iEyQAgKvQBb2m6WuxCRIAgABG61lvgnRzucIhAgACGK1nvQnSmZkShwgACGC0nvUmSNa3AgCch0lYrWQxw9l6EyTrWwEABDBMWc9wtt4EqWqbIwCA49EF3bL0tZ7hbL0JEquBAYAARsNsznBmmyMAgIYu6BaoO8NZ21YhNNS/pXvT6eIVITx7B7d9QtlG2xcACGA0yeYM56pB4v+R/7+UwQIA16ILugXszHCmDBYAgABuMTnDubYr2SPEgLfnwZ1+ymABAAjg1qs7w5kyWAAAArhJN5crpwtzJ85dOl2Yq22zWs9wpgwWAKAKk7BsqZo/5e/d9O/33BUc6teXuzqweyA01G9YCYsyWAAAAriZ9D1yLK9v85au3virNy781RsXqspdbejyaCuO9OavLpk9+QBlsADAleiCrsNw/pTGzkzmm8uV5175gdmtTz9MGSwAIIBRw2z+lGRnJrP1M2zt3cxBBgACGNXqzpCqO5OZGVgAgFouHQO22Dewis0ZUhY5ygwsAAABLIQQ2enZP/6Hs2b7BlYx20bQfo6yESEAoJYbu6C/9PL39IOylxeuf858IpVWZMOMp95MZjYiBAAQwELcmjlV5ff/7p/MJlLJIhsD3h7D9LWTo2xECACoTpBKxUWFiBcXF71e747fPd7VfXvtrU8FP/bF4C6zx8ph4+z07P8893bp6nvyl9bd14bPYGfgGQBg53q+sLDQ19dHAHd8APtu33TmD4K5H89bZyQ5CgAE8MpRCesD5Ws3PjHxWt3WrVm5KwAA7GMd8Ido6SvYrxcAQAvYgqqq+XxeCHH27NlEIqEoSqueuSKER4jxk9OhoX76mQEABPAHyuVyPp8fHR0VQmQymVAoVCgU6j7Ke9vGd5btZrCsckWfMwCgtTq7C3pqaioej8ufg8FgsVgsFot1H/Xsp3++ofYs1SIBAATwhwSDwVwup4WxEKK2C3ppaWlRRwgRGuqvWtfr791k8SpUiwQAtFzHjwEHAgH5QyKRSCaTtXeYmJgYHx+v+uWB3QOhoX5tNdHwR7f+6lf+gWqRAIA145B1wKlUSggRjUZrb1paWlpaWpI/Ly4u7tixw3Dd2Knzl48cywtdnSzZTU29KgBoQw5YB+yEZUiqqvr9fsP0FUJ0d3f36Zg9CdUiAQBrqeO7oOUapHA4LNvBBw8e9Pl8zT1VVb80Va4AAKuns7ugi8Xi4OCg9k+fzzc/P29xfwd0WQAABKUo152iKK6qZQ0AcAxKUQIAQAADAEAAAwAAAhgAAAIYAAAQwAAAEMAAAIAABgCg/Wx01buVVTvkpoQAgM4lr+QdXYvJXQE8NzcnhNixYwefXQBwxlXd6/USwB3A7/cLId56663OPWFr9tVyx44dFy9epGg2B4pjxYFqTwsLC3fffbe8qhPAHaCrq0sI4fV6+WTbYb2BIzhQHCsOVJtc1Tv1P8/5AwCAAAYAwBU2PPvss+56wxs2/Nqv/drGjRs59xwoDhTHigPFgVpHHvbTBQBg7dEFDQAAAQwAAAEMAAAIYGCdFYvFycnJTCYzOTlZLpct7hmLxazvAKAhmUxm5X+Y7caxk7CKxWImk1EUpVgsRqNRn8/X6B3cEyrWx0FV1Xw+L4Q4e/ZsIpFQFMW1l4Dh4eFcLieEKJfLkUgkm80a3k1V1UgkksvlXHus7PxxZTKZUqkkyxiFw2EOlNllSlVVIUShUDh06FAgEHBn9JZKpVgsNj8/b/hZsvmH2Y4qDhUIBOQP8/PzwWCwiTu4hPVxmJ+fTyQS8ud0Oq0oimsPVC6X045VpVLx+XyFQqH2bvPz89lsVlEUw1v5UEnJZFJ+rgqFgps/VHUP1OjoqPZzOByuuJgQYn5+vuk/zPbkzC5o2WKTfD7f1NRUsVhs6A4uUfc4TE1NxeNx+XMwGCwWi+48UPJQ6KvO+v1+w0Nx/PjxYDDo5q5CO39c8Xh8dHRUCKEoimy7cKAMD1Qmk9F+2dFFj9f9D7M9dbnzlHT0OVvLAxUMBrXr49TUlLxiuvPvvGpsyefz1Y42qap68OBBLojWHyoZPKqqqqoaj8dLpRIHyuwqFIvFBgcH4/F4KpXSvgqj0T/MtrXRnaeko8/ZGn92tWGnRCKRTCZd+3fu8/n0UVEul6uGo+Shc+1kAvsfqqmpqXK5rCiKoigjIyPDw8OFQoEDZfjXF41G5+bmVFXNZDIjIyPEbRN/mLSA2+6UdPQ5W5fPbiqVikQi0WjUtX/nVZe/UqlU1RmgqmqxWEylUqlUSk6u0fcx8qHSKIri8/nk0fP5fMVikQNl9pUuHo8nEolcLhePx/fv38/U+ib+MAngtjslHX3O1v6zq6qq3+93c/rKngDt8qc14IQQ+Xxe9hyGw+HoLfKf7pyzWvdDpSgKQWLzK10oFNKawtFoVA4DQf93Z/aHSQC377Wyo8/ZWh4ocWvETi4USaVSbr50ptNpudxwYmIinU7LX05MTOhXKJbL5cnJSSFEIpFw58SCuh8qRVECgYD8uVgsyn9yoAwP1NmzZy0y2yVUVZV/UxMTE3JRVtXfneEfZkdw7DrgfD6vqqr8BMdiMfnJjkQiDzzwgJx+aXgHd36RtDhQxWJxcHBQu7PP55ufn+fbN1b411csFpPJ5ODgoOxc5a/P7EDJWdByeDgYDLrzm4qDsRsSAADrgFKUAAAQwAAAEMAAAIAABgCAAAYAAAQwAAAEMAAAIIABACCAAQAAAQw4Sjwej8Vi8udMJqOV8gfQhihFCTjK8PDw2NhYIBAIhUK5XI79iYG2tZFDADjJa6+9Njw87PP5kskk6Qu0M7qgAUfx+Xxy78hgMMjRAAhgAGukWCyqqhoMBuUWqgDaFmPAgKMMDg5ms1lFUYaHh48ePcoOsgAtYACrLhKJaPu6p9PpSCRSLpc5LAAtYAAAQAsYAAACGAAAAhgAABDAAAAQwAAAgAAGAIAABgAABDAAAAQwAAAggAEAIIABACCAAQAAAQwAAAEMAAAIYAAAOtX/B5Guzvz7FyNxAAAAAElFTkSuQmCC" />

# Toy classification datasets

It’s a little trickier to create a synthetic classification dataset. Logically, we want two distinct classes of points, which are easily separated. Suppose that the dataset consists of only two types of points, (–1, –1) and (1, 1). Then a learning algorithm would have to learn a rule that separates these two data values.

$$ y_0 = (-1, -1) $$

$$ y_1 = (1, 1) $$

As before, let’s make the challenge a little more difficult by adding some Gaussian noise to both types of points:

$$ y_0 = (-1, -1) + N(0, \epsilon) $$

$$ y_1 = (1, 1) + N(0, \epsilon) $$

However, there’s a slight bit of trickiness here. Our points are two-dimensional, while the Gaussian noise we introduced previously is one-dimensional. Luckily, there exists a multivariate extension of the Gaussian. We won’t discuss the intricacies of the multivariate Gaussian here, but you do not need to understand the intricacies to follow our discussion.

The NumPy code to generate the synthetic dataset in Example 3-3 is slightly trickier than that for the linear regression problem since we have to use the stacking function np.vstack to combine the two different types of datapoints and associate them with different labels. (We use the related function np.concatenate to combine the one-dimensional labels.)

Example 3-3. Sample a toy classification dataset with NumPy

In [ ]:
# Generate synthetic data
N = 100
# Zeros form a Gaussian centered at (-1, -1)
# epsilon is .1
x_zeros = np.random.multivariate_normal(
    mean=np.array((-1, -1)), cov=.1*np.eye(2), size=(N/2,))
y_zeros = np.zeros((N/2,))
# Ones form a Gaussian centered at (1, 1)
# epsilon is .1
x_ones = np.random.multivariate_normal(
    mean=np.array((1, 1)), cov=.1*np.eye(2), size=(N/2,))
y_ones = np.ones((N/2,))
x_np = np.vstack([x_zeros, x_ones])
y_np = np.concatenate([y_zeros, y_ones])

Figure 3-7 plots the data generated by this code with Matplotlib to verify that the distribution is as expected. We see that the data resides in two classes that are neatly separated.

> **Figure 3-7: _Plot of the toy classification data distribution._**<br><img width="300" src="data:image/png;base64, ..." />

# New TensorFlow Concepts

Creating simple machine learning systems in TensorFlow will require that you learn some new TensorFlow concepts.

# Placeholders

A placeholder is a way to input information into a TensorFlow computation graph. Think of placeholders as the input nodes through which information enters TensorFlow. The key function used to create placeholders is tf.placeholder (Example 3-4).

Example 3-4. Create a TensorFlow placeholder

In [ ]:
tf.placeholder(tf.float32, shape=(2,2))

Output:<br>`<tf.Tensor 'Placeholder:0' shape=(2, 2) dtype=float32>`

We will use placeholders to feed datapoints x and labels y to our regression and classification algorithms.

# Feed dictionaries and Fetches

Recall that we can evaluate tensors in TensorFlow by using sess.run(var). How do we feed in values for placeholders in our TensorFlow computations then? The answer is to construct feed dictionaries. Feed dictionaries are Python dictionaries that map TensorFlow tensors to np.ndarray objects that contain the concrete values for these placeholders. A feed dictionary is best viewed as an input to a TensorFlow computation graph. What then is an output? TensorFlow calls these outputs fetches. You have seen fetches already. We used them extensively in the previous chapter without calling them as such; the fetch is a tensor (or tensors) whose value is retrieved from the computation graph after the computation (using placeholder values from the feed dictionary) is run to completion (Example 3-5).

Example 3-5. Using fetches

In [ ]:
a = tf.placeholder(tf.float32, shape=(1,))
b = tf.placeholder(tf.float32, shape=(1,))
c = a + b
with tf.Session() as sess:
    c_eval = sess.run(c, {a: [1.], b: [2.]})
    print(c_eval)

Output:<br>`[ 3.]`

# Name scopes

In complicated TensorFlow programs, there will be many tensors, variables, and placeholders defined throughout the program. tf.name_scope(name) provides a simple scoping mechanism for managing these collections of variables (Example 3-6). All computational graph elements created within the scope of a tf.name_scope(name) call will have name prepended to their names.

This organizational tool is most useful when combined with TensorBoard, since it aids the visualization system in automatically grouping graph elements within the same name scope. You will learn more about TensorBoard further in the next section.

Example 3-6. Using namescopes to organize placeholders

In [ ]:
N = 5
with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (N, 1))
    y = tf.placeholder(tf.float32, (N,))
x

Output:<br>`<tf.Tensor 'placeholders/Placeholder:0' shape=(5, 1) dtype=float32>`

# Optimizers

The primitives introduced in the last two sections already hint at how machine learning is done in TensorFlow. You have learned how to add placeholders for datapoints and labels and how to use tensorial operations to define the loss function. The missing piece is that you still don’t know how to perform gradient descent using TensorFlow.

While it is in fact possible to define optimization algorithms such as gradient descent directly in Python using TensorFlow primitives, TensorFlow provides a collection of optimization algorithms in the tf.train module. These algorithms can be added as nodes to the TensorFlow computation graph.

<div style="border: 1px solid black; padding: 10px;"><b style="font-size: 2em;">Which optimizer should I use?</b><br> 
There are many possible optimizers available in tf.train. For a short preview, this list includes tf.train.GradientDescentOptim izer, tf.train.MomentumOptimizer, tf.train.AdagradOptim izer, tf.train.AdamOptimizer, and many more. What’s the difference between these various optimizers?
<br>
Almost all of these optimizers are based on the idea of gradient descent. Recall the simple gradient descent rule we previously introduced:
$$ W = W - \alpha \nabla W $$
Mathematically, this update rule is primitive. There are a variety of mathematical tricks that researchers have discovered that enable faster optimization without using too much extra computation. In general, tf.train.AdamOptimizer is a good default that is relatively robust. (Many optimizer methods are very sensitive to hyperparameter choice. It’s better for beginners to avoid trickier methods until they have a good grasp of the behavior of different optimization algorithms.)
        </div>

Example 3-7 is a short bit of code that adds an optimizer to the computation graph that minimizes a predefined loss l.

Example 3-7. Adding an Adam optimizer to TensorFlow computation graph

In [ ]:
learning_rate = .001
with tf.name_scope("optim"):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(l)

# Taking gradients with TensorFlow

We mentioned previously that it is possible to directly implement gradient descent algorithms in TensorFlow. While most use cases don’t need to reimplement the contents of tf.train, it can be useful to look at gradient values directly for debugging purposes. tf.gradients provides a useful tool for doing so (Example 3-8).

Example 3-8. Taking gradients directly

In [ ]:
W = tf.Variable((3,))
l = tf.reduce_sum(W)
gradW = tf.gradients(l, W)
gradW

Output:<br>`[<tf.Tensor 'gradients/Sum_grad/Tile:0' shape=(1,) dtype=int32>]`

This code snippet symbolically pulls down the gradients of loss l with respect to learnable parameter (tf.Variable) W. tf.gradients returns a list of the desired gradients. Note that the gradients are themselves tensors! TensorFlow performs symbolic differentiation, which means that gradients themselves are parts of the computational graph. One neat side effect of TensorFlow’s symbolic gradients is that it’s possible to stack derivatives in TensorFlow. This can sometimes be useful for more advanced algorithms.

# Summaries and file writers for TensorBoard

Gaining a visual understanding of the structure of a tensorial program can be very useful. The TensorFlow team provides the TensorBoard package for this purpose. TensorBoard starts a web server (on localhost by default) that displays various useful visualizations of a TensorFlow program. However, in order for TensorFlow programs to be inspected with TensorBoard, programmers must manually write logging statements. tf.train.FileWriter() specifies the logging directory for a TensorBoard program and tf.summary writes summaries of various TensorFlow variables to the specified logging directory. In this chapter, we will only use tf.summary.scalar, which summarizes a scalar quantity, to track the value of the loss function. tf.summary.merge_all() is a useful logging aid that merges multiple summaries into a single summary for convenience.

The code snippet in Example 3-9 adds a summary for the loss and specifies a logging directory.

Example 3-9. Adding a summary for the loss

In [ ]:
with tf.name_scope("summaries"):
    tf.summary.scalar("loss", l)
    merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('/tmp/lr-train', tf.get_default_graph())

# Training models with TensorFlow

Suppose now that we have specified placeholders for datapoints and labels, and have defined a loss with tensorial operations. We have added an optimizer node train_op to the computational graph, which we can use to perform gradient descent steps (while we may actually use a different optimizer, we will refer to updates as gradient descent for convenience). How can we iteratively perform gradient descent to learn on this dataset?

The simple answer is that we use a Python for-loop. In each iteration, we use sess.run() to fetch the train_op along with the merged summary op merged and the loss l from the graph. We feed all datapoints and labels into sess.run() using a feed dictionary.

The code snippet in Example 3-10 demonstrates this simple learning method. Note that we don’t make use of minibatches for pedagogical simplicity. Code in following chapters will use minibatches when training on larger datasets.

Example 3-10. A simple example of training a model

In [ ]:
n_steps = 1000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Train model
    for i in range(n_steps):
        feed_dict = {x: x_np, y: y_np}
        _, summary, loss = sess.run([train_op, merged, l], feed_dict=feed_dict)
        print("step %d, loss: %f" % (i, loss))
        train_writer.add_summary(summary, i)